# CAR DETECTION ALONG WITH CLASSIFICATION

In [ ]:
# from keras.models import Sequential, Model
from keras.layers import Input  #, Reshape, Activation, Conv2D,MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
# from keras.layers.advanced_activations import LeakyReLU
# from keras.optimizers import SGD, Adam, RMSprop
# from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
# import tensorflow as tf
import pandas as pd
# import imgaug as ia
# from tqdm import tqdm
from console_progressbar import ProgressBar
# from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2, time
from PIL import Image
from utils_ import  decode_netout, draw_boxes, boxes_make,load_model, final_image,crop_blob
from architecture import architecture
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

%matplotlib inline

In [ ]:
LABELS = ['car']

IMAGE_H, IMAGE_W = 416, 416 #based on the coco dataset for detection training
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
CLASS            = 1
OBJ_THRESHOLD    = 0.3
NMS_THRESHOLD    = 0.3
ANCHORS          = [0.47,0.42, 1.28,1.09, 2.68,2.54, 6.17,4.34, 10.56,9.29]

TRUE_BOX_BUFFER  = 50

## Model Architecture for YOLO

In [ ]:
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3))
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4))

model = architecture(input_image,true_boxes)

In [ ]:
model.load_weights("weights_coco_car_1.h5") #weight for car detection --> pretrained wt using COCO car dataset

In [ ]:
path_val = './sample/'

In [ ]:
imag_val=[]
image_name = []  ####

for file in os.listdir(path_val):
    if file.endswith('.jpg'):
        cc = os.path.join(file)
        i_n = file.strip('.jpg') ###
        image_name.append(i_n) ####
        imag_val.append(cc)



In [ ]:
for m in range(len(imag_val)):
    
    image = cv2.imread(path_val + imag_val[m])
    dummy_array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))


    input_image = cv2.resize(image, (416, 416))
    input_image = input_image / 255.
    input_image = input_image[:,:,::-1]
    input_image = np.expand_dims(input_image, 0)

    netout = model.predict([input_image, dummy_array])

    boxes = decode_netout(netout[0], 
                      obj_threshold=OBJ_THRESHOLD,
                      nms_threshold=NMS_THRESHOLD,
                      anchors=ANCHORS, 
                      nb_class=CLASS)          #create the boxes
#       
    b = boxes_make(image, boxes) # taking the corresponding blobes of each image
    
    
    if len(b)>0:
        current_loc = os.getcwd()
        new_dir = 'save_' + image_name[m]
        os.mkdir(new_dir)
        
        for j in range(len(b)):
              
            pat = path_val + imag_val[m]
            save_loc = current_loc + new_dir

            crop_blob(pat, (b[j][0], b[j][1], b[j][2], b[j][3]), save_loc + str(j) + '_' + imag_val[m])


In [ ]:
img_test=[]
p = '/home/abhijithmtmt17/essi/CAR_DETECTIONMODEL/sample/save/'
for file in os.listdir(p):
    if file.endswith('.jpg'):
        cc = os.path.join(file)
        img_test.append(cc)


In [ ]:
for i in range(len(img_test)):
    
    image_ = cv2.imread(p + img_test[i])
    inputimage = cv2.resize(image_, (224,224))
    inputimage = inputimage[:,:,::-1]

    ii = './sample/save/' + img_test[i]
    plt.imsave(ii, inputimage[:,:])

#     plt.axis('off')
    
#     plt.savefig(ii)
    


## TEST USING CAR CLASSIFICATION MODEL

In [ ]:
path = './sample/save/'
test_image = []

import os
for file in os.listdir(path):
    if file.endswith('.jpg'):
        cc = os.path.join(file)
        test_image.append(cc)

In [ ]:
model = load_model() #model trained on standford car dataset

pb = ProgressBar(total=100, prefix='Predicting test data', suffix='', decimals=3, length=50, fill='=')
num_samples = len(test_image)
i=0

# start = time.time()
# l = str(m)
out = open('result_' + imag_val[m] + '.txt', 'a')
for i in range(num_samples):
    filename = os.path.join(path,test_image[i])
    bgr_img = cv2.imread(filename)
    rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    rgb_img = cv2.resize(rgb_img, (224,224))
    rgb_img = np.expand_dims(rgb_img, 0)
    preds = model.predict(rgb_img)
    prob = np.max(preds)
    
    
    if prob > 0.4:
        print(prob)
        class_id = np.argmax(preds)
        name = filename.strip('/home/abhijithmtmt17/essi/CAR_DETECTIONMODEL/sample/')
        out.write(test_image[i] + '\t{}\n'.format(str(class_id + 1)))
#     pb.print_progress_bar((i + 1) * 100 / num_samples)

# end = time.time()
# seconds = end - start
# print('avg fps: {}'.format(str(num_samples / seconds)))

out.close()
K.clear_session()



In [ ]:
imag_val[0]

In [ ]:
with open('result_' + imag_val[m] + '.txt', 'r') as f:
    pred_class = [line.strip() for line in f] #predclass have category no.

imagename = [i.split('\t', 1)[0] for i in pred_class]
cat_name = [i.split('\t', 1)[1] for i in pred_class]

In [ ]:
model_name = pd.read_csv('model_names.csv')
model_name_list=[]

for i in range(len(model_name)):
    
    model_name_list.append(model_name.iloc[i].values)

In [ ]:
box_no = []
model = []
for x in range(len(cat_name)):

    cat = int(cat_name[x]) #converting the class no to integer
    image = imagename[x] 
    
    for i in range(len(model_name)):
        
        if cat == (i+1):

#             print(image + ' - ' + cat_name[x] + ' --> '+ model_name_list[i]) #
            k = image.rsplit('_',2)
            box_no.append(k[0])
            model.append(model_name_list[i])
            
        

In [ ]:
boxes_list = [] #taking corresponding blob parameters

for val in box_no:
    y = int(val)
    boxes_list.append(b[y])

In [ ]:
#printing final image

image = cv2.imread(path_val + imag_val[m])
plt.figure(figsize=(10,10))
image = final_image(image, boxes_list, model)
plt.imshow(image[:,:,::-1]); plt.show()
final_save_loc = './sample/save_final/' + imag_val[m]
plt.imsave(final_save_loc, image[:,:,::-1])


In [ ]:
one = os.getcwd()

one


In [ ]:
fld ='try'

os.mkdir(fld)

In [ ]:
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)


In [ ]:
psss = one + '/' + fld + '/'

psss